In [ ]:
from pymatgen.io.vasp import Poscar, Xdatcar
from pymatgen.symmetry.groups import SpaceGroup
from pymatgen import Lattice
import numpy as np
import operator
from site_analysis import Atom, Analysis, ShortestDistanceSite, get_vertex_indices, AtomsTrajectory, SitesTrajectory
from collections import Counter
import tqdm
from site_analysis import PolyhedralSite

In [ ]:
directory = '/Users/bjm42/Documents/Work/Projects/Active Projects/argyrodites/4a-4c_disorder/data'
structure = Poscar.from_file(f'{directory}/reference_structures/Li6PS5I_alltet_sites.POSCAR.vasp').structure

In [7]:
from pymatgen import Structure, Lattice
lattice = structure.lattice
t0 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['P'], coords=[[0.5, 0.0, 0.0]])
t1 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.9, 0.9, 0.6]])
t2 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.25, 0.433, 0.933]])
t3 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.25, 0.25, 0.25]])
t4 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.15, 0.15, 0.15]])
t5 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['Li'], coords=[[0.0, 0.183, 0.183]])
i1 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['I'], coords=[[0.0, 0.0, 0.0]])
s2 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['S'], coords=[[0.25, 0.25, 0.75]])
s1 = Structure.from_spacegroup(sg='F-43m', lattice=lattice, species=['S'], coords=[[0.38368, 0.11632, 0.11632]])
tet_sites = [t0, t1, t2, t3, t4, t5]

In [8]:
sublattices = [ t1, t2, t3, t4, t5, i1, s2, s1 ]
species = ['Cs', 'Na', 'Rb', 'K', 'Li', 'I', 'S', 'S']
structure = t0
for tet, spec in zip( sublattices, species ):
    for s in tet:
        structure.append(spec, coords=s.frac_coords, coords_are_cartesian=False)

In [9]:
structure

Structure Summary
Lattice
    abc : 10.1413497925 10.1413497925 10.1413497925
 angles : 90.0 90.0 90.0
 volume : 1043.0071548006667
      A : 10.1413497925 0.0 0.0
      B : 0.0 10.1413497925 0.0
      C : 0.0 0.0 10.1413497925
PeriodicSite: P (5.0707, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: P (0.0000, 5.0707, 0.0000) [0.0000, 0.5000, 0.0000]
PeriodicSite: P (0.0000, 0.0000, 5.0707) [0.0000, 0.0000, 0.5000]
PeriodicSite: P (5.0707, 5.0707, 5.0707) [0.5000, 0.5000, 0.5000]
PeriodicSite: Cs (9.1272, 9.1272, 6.0848) [0.9000, 0.9000, 0.6000]
PeriodicSite: Cs (9.1272, 1.0141, 4.0565) [0.9000, 0.1000, 0.4000]
PeriodicSite: Cs (1.0141, 1.0141, 6.0848) [0.1000, 0.1000, 0.6000]
PeriodicSite: Cs (1.0141, 9.1272, 4.0565) [0.1000, 0.9000, 0.4000]
PeriodicSite: Cs (6.0848, 9.1272, 9.1272) [0.6000, 0.9000, 0.9000]
PeriodicSite: Cs (9.1272, 4.0565, 1.0141) [0.9000, 0.4000, 0.1000]
PeriodicSite: Cs (4.0565, 1.0141, 9.1272) [0.4000, 0.1000, 0.9000]
PeriodicSite: Cs (1.0141, 6.0848, 1.014

In [10]:
structure

Structure Summary
Lattice
    abc : 10.1413497925 10.1413497925 10.1413497925
 angles : 90.0 90.0 90.0
 volume : 1043.0071548006667
      A : 10.1413497925 0.0 0.0
      B : 0.0 10.1413497925 0.0
      C : 0.0 0.0 10.1413497925
PeriodicSite: P (5.0707, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: P (0.0000, 5.0707, 0.0000) [0.0000, 0.5000, 0.0000]
PeriodicSite: P (0.0000, 0.0000, 5.0707) [0.0000, 0.0000, 0.5000]
PeriodicSite: P (5.0707, 5.0707, 5.0707) [0.5000, 0.5000, 0.5000]
PeriodicSite: Cs (9.1272, 9.1272, 6.0848) [0.9000, 0.9000, 0.6000]
PeriodicSite: Cs (9.1272, 1.0141, 4.0565) [0.9000, 0.1000, 0.4000]
PeriodicSite: Cs (1.0141, 1.0141, 6.0848) [0.1000, 0.1000, 0.6000]
PeriodicSite: Cs (1.0141, 9.1272, 4.0565) [0.1000, 0.9000, 0.4000]
PeriodicSite: Cs (6.0848, 9.1272, 9.1272) [0.6000, 0.9000, 0.9000]
PeriodicSite: Cs (9.1272, 4.0565, 1.0141) [0.9000, 0.4000, 0.1000]
PeriodicSite: Cs (4.0565, 1.0141, 9.1272) [0.4000, 0.1000, 0.9000]
PeriodicSite: Cs (1.0141, 6.0848, 1.014

In [11]:
structure.to(filename="test.POSCAR")

In [ ]:
anion_sites = [ s for s in md_structure if s.species_string in ['S', 'I'] ]
struc = Structure.from_sites(anion_sites)
for s in t0_sites:
    s.get_vertex_coords(structure=md_structure)
    struc.append('P', coords=s.centre(), coords_are_cartesian=False)
for s in t1_sites:
    s.get_vertex_coords(structure=md_structure)
    struc.append('Rb', coords=s.centre(), coords_are_cartesian=False)
for s in t2_sites:
    s.get_vertex_coords(structure=md_structure)
    struc.append('Na', coords=s.centre(), coords_are_cartesian=False)
for s in t3_sites:
    s.get_vertex_coords(structure=md_structure)
    struc.append('Cs', coords=s.centre(), coords_are_cartesian=False)
for s in t4_sites:
    s.get_vertex_coords(structure=md_structure)
    struc.append('K', coords=s.centre(), coords_are_cartesian=False)
for s in t5_sites:
    s.get_vertex_coords(structure=md_structure)
    struc.append('Li', coords=s.centre(), coords_are_cartesian=False)
struc.to(filename='test.POSCAR')   

In [ ]:
# Can we process MD with this?

In [ ]:
sites = t0_sites + t1_sites + t2_sites + t3_sites + t4_sites + t5_sites
atoms = [Atom(species_string='Li') for site in md_structure if site.species_string is 'Li']

In [ ]:
analysis = Analysis(atoms=atoms, sites=sites)

In [ ]:
analysis.append_timestep(md_structure)

In [ ]:
analysis.at

In [ ]:
from scipy.spatial.qhull import ConvexHull

In [ ]:
vols = []
for label in ['0', '1', '2', '3', '4', '5']:
    these_sites = [ s for s in sites if s.label is label  ]
    vols.append( [ ConvexHull(s.vertex_coords).volume for s in these_sites ])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(vols[0], bins=30, alpha=0.7, label='0')
plt.hist(vols[1], bins=30, alpha=0.7, label='1')
plt.hist(vols[2], bins=30, alpha=0.7, label='2')
plt.hist(vols[3], bins=30, alpha=0.7, label='3')
plt.hist(vols[4], bins=30, alpha=0.7, label='4')
plt.hist(vols[5], bins=30, alpha=0.7, label='5')
plt.legend()
plt.show()

In [ ]:
from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import AbstractGeometry, symmetry_measure
from pymatgen.analysis.chemenv.coordination_environments.coordination_geometries import AllCoordinationGeometries
from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import symmetry_measure

In [ ]:
from polyhedral_analysis.symmetry_measure import SymmetryMeasure

In [ ]:
tet_sm = SymmetryMeasure.from_name('Tetrahedron')

In [ ]:
sms = []
for label in ['0', '1', '2', '3', '4', '5']:
    sms.append([])
    these_sites = [ s for s in analysis.sites if s.label == label ]
    for site in these_sites:
        ag = AbstractGeometry( bare_coords=list(site.vertex_coords), 
                               central_site=site.centre())
        sms[-1].append(tet_sm.minimum_symmetry_measure(ag))

In [ ]:
plt.hist(sms[0], bins=30, alpha=0.7, label='0')
plt.hist(sms[1], bins=30, alpha=0.7, label='1')
plt.hist(sms[2], bins=30, alpha=0.7, label='2')
plt.hist(sms[3], bins=30, alpha=0.7, label='3')
plt.hist(sms[4], bins=30, alpha=0.7, label='4')
plt.hist(sms[5], bins=30, alpha=0.7, label='5')
plt.legend()
plt.show()

In [ ]:
xdatcar = Xdatcar(f'{directory}/Li6PS5Cl/50p/run1/inherent_XDATCAR.gz')
md_structure = xdatcar.structures[0]

In [ ]:
list(site.vertex_coords)

In [ ]:
analysis.reset()

xdatcar = Xdatcar(f'{directory}/Li6PS5I/0p/run1/inherent_XDATCAR.gz')

analysis.trajectory_from_structures( xdatcar.structures, progress='notebook')

In [ ]:
c = Counter()
for site in analysis.sites:
    c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
for label, n in c.items():
    print(f'{label}: {n}')

In [ ]:
analysis.reset()

xdatcar = Xdatcar(f'{directory}/Li6PS5Cl/0p/run1/inherent_XDATCAR.gz')

analysis.trajectory_from_structures( xdatcar.structures, progress='notebook')

In [ ]:
c = Counter()
for site in analysis.sites:
    c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
for label, n in c.items():
    print(f'{label}: {n}')

In [ ]:
md_structure = Poscar.from_file('./Li6PS5Cl_222_POSCAR').structure

In [ ]:
t0_indices = get_nearest_n_site_indices( md_structure, t0*[2,2,2], vertex_species=['S', 'I'], n_coord=4)
t1_indices = get_nearest_n_site_indices( md_structure, t1*[2,2,2], vertex_species=['S', 'I'], n_coord=4)
t2_indices = get_nearest_n_site_indices( md_structure, t2*[2,2,2], vertex_species=['S', 'I'], n_coord=4)
t3_indices = get_nearest_n_site_indices( md_structure, t3*[2,2,2], vertex_species=['S', 'I'], n_coord=4)
t4_indices = get_nearest_n_site_indices( md_structure, t4*[2,2,2], vertex_species=['S', 'I'], n_coord=4)
t5_indices = get_nearest_n_site_indices( md_structure, t5*[2,2,2], vertex_species=['S', 'I'], n_coord=4)

In [ ]:
t0_sites = [ PolyhedralSite(vertex_species=['S','Cl'], vertex_indices=vi, 
             label='0') for vi in t0_indices ]
t1_sites = [ PolyhedralSite(vertex_species=['S','Cl'], vertex_indices=vi, 
             label='1') for vi in t1_indices ]
t2_sites = [ PolyhedralSite(vertex_species=['S','Cl'], vertex_indices=vi, 
             label='2') for vi in t2_indices ]
t3_sites = [ PolyhedralSite(vertex_species=['S','Cl'], vertex_indices=vi, 
             label='3') for vi in t3_indices ]
t4_sites = [ PolyhedralSite(vertex_species=['S','Cl'], vertex_indices=vi, 
             label='4') for vi in t4_indices ]
t5_sites = [ PolyhedralSite(vertex_species=['S','Cl'], vertex_indices=vi, 
             label='5') for vi in t5_indices ]

In [ ]:
t0_indices

In [ ]:
sites = t0_sites + t1_sites + t2_sites + t3_sites + t4_sites + t5_sites
atoms = [Atom(species_string='Li') for site in md_structure if site.species_string is 'Li']

In [ ]:
analysis = Analysis(atoms=atoms, sites=sites)

In [ ]:
analysis.analyse_structure(md_structure)

In [ ]:
analysis.reset()

xdatcar = Xdatcar(f'{directory}/Li6PS5Cl/0p/run1/inherent_XDATCAR.gz')

analysis.trajectory_from_structures( xdatcar.structures, progress='notebook')

In [ ]:
c = Counter()
for site in analysis.sites:
    c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
for label, n in c.items():
    print(f'{label}: {n}')

In [ ]:
analysis.reset()

xdatcar = Xdatcar(f'{directory}/Li6PS5Cl/100p/run1/inherent_XDATCAR.gz')

analysis.trajectory_from_structures( xdatcar.structures, progress='notebook')

In [ ]:
c = Counter()
for site in analysis.sites:
    c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
for label, n in c.items():
    print(f'{label}: {n}')

In [ ]:
analysis.reset()

xdatcar = Xdatcar(f'{directory}/Li6PS5Cl/0p/run1/inherent_XDATCAR.gz')

analysis.trajectory_from_structures( xdatcar.structures, progress='notebook')

In [ ]:
c = Counter()
for site in analysis.sites:
    c[site.label] += len([ 1 for ts in site.trajectory if len(ts)>0 ])
for label, n in c.items():
    print(f'{label}: {n}')